<a href="https://colab.research.google.com/github/ferranfont/Introduccion_al_trading_algoritmico/blob/main/Quant_momentum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**QUANTITATIVE MOMENTUM SYSTEM**

In [115]:
import pandas as pd
import numpy as np
import requests
import matplotlib
import math
from scipy import stats # Esta libreria es nueva y la usaremos para calcular el percentil

In [116]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [117]:
# Importamos todos los tickets del SP500
stocks = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [118]:
# Hacemos un API a un end point de momentum
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'
symbol='BA'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avg10Volume': 7982876,
 'avg30Volume': 10391389,
 'beta': 1.376984479293487,
 'companyName': 'Boeing Co.',
 'day200MovingAvg': 218.7,
 'day30ChangePercent': -0.02506494327631323,
 'day50MovingAvg': 221.3,
 'day5ChangePercent': -0.016132696937991645,
 'dividendYield': 0,
 'employees': 141490,
 'exDividendDate': '',
 'float': 0,
 'marketcap': 121921276372,
 'maxChangePercent': 2.189964021211343,
 'month1ChangePercent': -0.03879065401201946,
 'month3ChangePercent': -0.024562274939314215,
 'month6ChangePercent': -0.01485694268404653,
 'nextDividendDate': '',
 'nextEarningsDate': '',
 'peRatio': -14.986559387029237,
 'sharesOutstanding': 605238105,
 'ttmDividendRate': 0,
 'ttmEPS': -14.6,
 'week52change': -0.03974029059794781,
 'week52high': 291.2,
 'week52highSplitAdjustOnly': 286.87,
 'week52low': 188.56,
 'week52lowSplitAdjustOnly': 185.97,
 'year1ChangePercent': -0.04058794004285209,
 'year2ChangePercent': -0.3921658584753035,
 'year5ChangePercent': 0.296797246512869,
 'ytdChangePerce

In [119]:
# Consultamos un campo en particular
data['month1ChangePercent']*100

-3.879065401201946

In [120]:
round(data['peRatio'],1)

-15.0

In [121]:
my_colums = ['Ticker','Descripción', 'Rentab_1año','Rentab_6m','Rentab_1m','PER','n_acciones_a_comprar']
final_dataframe = pd.DataFrame(columns=my_colums)
final_dataframe

,Ticker,Descripción,Rentab_1año,Rentab_6m,Rentab_1m,PER,n_acciones_a_comprar


In [122]:
final_dataframe = final_dataframe.append(
                                        pd.Series([stocks, 
                                                   data['companyName'], 
                                                   round(data['year1ChangePercent'],2),
                                                   round(data['month6ChangePercent'],2),
                                                   round(data['month1ChangePercent'],2),
                                                   round(data['peRatio'],1),
                                                   'N/A'],  
                                                  index =my_colums), 
                                        ignore_index = True)
final_dataframe

,Ticker,Descripción,Rentab_1año,Rentab_6m,Rentab_1m,PER,n_acciones_a_comprar
0,Ticker 0 A 1 AAL 2 AAP 3 ...,Boeing Co.,-0.04,-0.01,-0.04,-15.0,N/A


In [123]:
# Rellenamos el dataframe en cada una de sus filas (stocks tickers) con el valor que obtenemos de la API call
for i in stocks['Ticker'][:80]: #para no ralentizar el proceso lo haremos tan sólo para las primeros 80 símbolos [:80]
  api_url = f'https://sandbox.iexapis.com/stable/stock/{i}/stats?token=Tpk_059b97af715d417d9f49f50b51b1c448'
  data = requests.get(api_url).json()
  final_dataframe = final_dataframe.append(
                                        pd.Series([i, 
                                                   data['companyName'], 
                                                   round(data['year1ChangePercent'],2),
                                                   round(data['month6ChangePercent'],2),
                                                   round(data['month1ChangePercent'],2),
                                                   round(data['peRatio'],1),
                                                   'N/A'], 
                                                  index =my_colums), 
                                        ignore_index = True)

In [124]:
final_dataframe

,Ticker,Descripción,Rentab_1año,Rentab_6m,Rentab_1m,PER,n_acciones_a_comprar
0,Ticker 0 A 1 AAL 2 AAP 3 ...,Boeing Co.,-0.04,-0.01,-0.04,-15.0,N/A
1,A,Agilent Technologies Inc.,0.05,-0.22,-0.06,17.2,N/A
2,AAL,American Airlines Group Inc,-0.04,-0.05,0.03,-3.7,N/A
3,AAP,Advance Auto Parts Inc,0.35,0.06,-0.07,11.0,N/A
4,AAPL,Apple Inc,0.30,0.15,0.01,28.4,N/A
...,...,...,...,...,...,...,...
76,C,Citigroup Inc,0.00,-0.07,0.01,6.1,N/A
77,CAG,Conagra Brands Inc,0.09,0.09,0.01,16.4,N/A
78,CAH,"Cardinal Health, Inc.",0.08,0.08,0.04,28.0,N/A
79,CARR,Carrier Global Corp,0.18,-0.21,-0.10,11.9,N/A


In [125]:
# Vamos a retirar los que tienen bajo momentum con el método .sort_value()
final_dataframe.sort_values('Rentab_1año', ascending=False, inplace=True)
final_dataframe.reset_index(inplace=True) #reseteamos el índice de nuevo
final_dataframe[:30]


,index,Ticker,Descripción,Rentab_1año,Rentab_6m,Rentab_1m,PER,n_acciones_a_comprar
0,43,APA,APA Corporation,0.68,1.04,-0.00,19.8,N/A
1,38,ANET,Arista Networks Inc,0.63,0.38,0.00,23.4,N/A
2,55,AZO,Autozone Inc.,0.60,0.18,-0.01,17.7,N/A
3,40,ANTM,Anthem Inc,0.55,0.20,-0.00,9.1,N/A
4,54,AXP,American Express Co.,0.50,0.23,0.21,9.7,N/A
5,19,AIG,American International Group Inc,0.50,0.19,0.05,2.9,N/A
6,20,AIV,Apartment Investment & Management Co.,0.46,0.09,-0.01,-36.2,N/A
7,5,ABBV,Abbvie Inc,0.45,0.26,0.07,34.1,N/A
8,12,ADM,Archer Daniels Midland Co.,0.41,0.31,0.09,8.0,N/A
9,50,AVB,Avalonbay Communities Inc.,0.37,0.07,-0.05,33.6,N/A


In [126]:
final_dataframe['Percentil_1año']=0
final_dataframe['Percentil_6meses']=0
final_dataframe['Percentil_1mes']=0
final_dataframe

,index,Ticker,Descripción,Rentab_1año,Rentab_6m,Rentab_1m,PER,n_acciones_a_comprar,Percentil_1año,Percentil_6meses,Percentil_1mes
0,43,APA,APA Corporation,0.68,1.04,-0.00,19.8,N/A,0,0,0
1,38,ANET,Arista Networks Inc,0.63,0.38,0.00,23.4,N/A,0,0,0
2,55,AZO,Autozone Inc.,0.60,0.18,-0.01,17.7,N/A,0,0,0
3,40,ANTM,Anthem Inc,0.55,0.20,-0.00,9.1,N/A,0,0,0
4,54,AXP,American Express Co.,0.50,0.23,0.21,9.7,N/A,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
76,49,ATVI,Activision Blizzard Inc,-0.20,-0.03,-0.01,24.4,N/A,0,0,0
77,17,AES,AES Corp.,-0.23,-0.09,-0.07,27.5,N/A,0,0,0
78,39,ANSS,Ansys Inc.,-0.25,-0.17,-0.12,56.2,N/A,0,0,0
79,63,BIIB,Biogen Inc,-0.25,-0.39,-0.09,9.9,N/A,0,0,0


In [127]:
n_valores = 30

In [128]:
for i in range(0,n_valores):
  final_dataframe.loc[i,'Percentil_1año'] = round(stats.percentileofscore(final_dataframe['Rentab_1año'],final_dataframe.loc[i,'Rentab_1año']),1)
  final_dataframe.loc[i,'Percentil_6meses'] = round(stats.percentileofscore(final_dataframe['Rentab_6m'],final_dataframe.loc[i,'Rentab_6m']),1)
  final_dataframe.loc[i,'Percentil_1mes'] = round(stats.percentileofscore(final_dataframe['Rentab_1m'],final_dataframe.loc[i,'Rentab_1m']),1)
final_dataframe[:30]

,index,Ticker,Descripción,Rentab_1año,Rentab_6m,Rentab_1m,PER,n_acciones_a_comprar,Percentil_1año,Percentil_6meses,Percentil_1mes
0,43,APA,APA Corporation,0.68,1.04,-0.00,19.8,N/A,100.0,100.0,69.1
1,38,ANET,Arista Networks Inc,0.63,0.38,0.00,23.4,N/A,98.8,97.5,69.1
2,55,AZO,Autozone Inc.,0.60,0.18,-0.01,17.7,N/A,97.5,86.4,59.9
3,40,ANTM,Anthem Inc,0.55,0.20,-0.00,9.1,N/A,96.3,89.5,69.1
4,54,AXP,American Express Co.,0.50,0.23,0.21,9.7,N/A,94.4,91.4,100.0
5,19,AIG,American International Group Inc,0.50,0.19,0.05,2.9,N/A,94.4,87.7,89.5
6,20,AIV,Apartment Investment & Management Co.,0.46,0.09,-0.01,-36.2,N/A,92.6,72.8,59.9
7,5,ABBV,Abbvie Inc,0.45,0.26,0.07,34.1,N/A,91.4,93.8,92.0
8,12,ADM,Archer Daniels Midland Co.,0.41,0.31,0.09,8.0,N/A,90.1,96.3,95.1
9,50,AVB,Avalonbay Communities Inc.,0.37,0.07,-0.05,33.6,N/A,86.4,66.0,35.8


In [129]:
final_dataframe['Media']=0
final_dataframe


,index,Ticker,Descripción,Rentab_1año,Rentab_6m,Rentab_1m,PER,n_acciones_a_comprar,Percentil_1año,Percentil_6meses,Percentil_1mes,Media
0,43,APA,APA Corporation,0.68,1.04,-0.00,19.8,N/A,100.0,100.0,69.1,0
1,38,ANET,Arista Networks Inc,0.63,0.38,0.00,23.4,N/A,98.8,97.5,69.1,0
2,55,AZO,Autozone Inc.,0.60,0.18,-0.01,17.7,N/A,97.5,86.4,59.9,0
3,40,ANTM,Anthem Inc,0.55,0.20,-0.00,9.1,N/A,96.3,89.5,69.1,0
4,54,AXP,American Express Co.,0.50,0.23,0.21,9.7,N/A,94.4,91.4,100.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
76,49,ATVI,Activision Blizzard Inc,-0.20,-0.03,-0.01,24.4,N/A,0.0,0.0,0.0,0
77,17,AES,AES Corp.,-0.23,-0.09,-0.07,27.5,N/A,0.0,0.0,0.0,0
78,39,ANSS,Ansys Inc.,-0.25,-0.17,-0.12,56.2,N/A,0.0,0.0,0.0,0
79,63,BIIB,Biogen Inc,-0.25,-0.39,-0.09,9.9,N/A,0.0,0.0,0.0,0


In [130]:
from statistics import mean
for i in range(0,n_valores):
  final_dataframe.loc[i,'Media'] = round(mean([final_dataframe.loc[i,'Percentil_6meses'],final_dataframe.loc[i,'Percentil_1año'],final_dataframe.loc[i,'Percentil_1mes']]),1)

final_dataframe.head(22)


,index,Ticker,Descripción,Rentab_1año,Rentab_6m,Rentab_1m,PER,n_acciones_a_comprar,Percentil_1año,Percentil_6meses,Percentil_1mes,Media
0,43,APA,APA Corporation,0.68,1.04,-0.00,19.8,N/A,100.0,100.0,69.1,89.7
1,38,ANET,Arista Networks Inc,0.63,0.38,0.00,23.4,N/A,98.8,97.5,69.1,88.5
2,55,AZO,Autozone Inc.,0.60,0.18,-0.01,17.7,N/A,97.5,86.4,59.9,81.3
3,40,ANTM,Anthem Inc,0.55,0.20,-0.00,9.1,N/A,96.3,89.5,69.1,85.0
4,54,AXP,American Express Co.,0.50,0.23,0.21,9.7,N/A,94.4,91.4,100.0,95.3
5,19,AIG,American International Group Inc,0.50,0.19,0.05,2.9,N/A,94.4,87.7,89.5,90.5
6,20,AIV,Apartment Investment & Management Co.,0.46,0.09,-0.01,-36.2,N/A,92.6,72.8,59.9,75.1
7,5,ABBV,Abbvie Inc,0.45,0.26,0.07,34.1,N/A,91.4,93.8,92.0,92.4
8,12,ADM,Archer Daniels Midland Co.,0.41,0.31,0.09,8.0,N/A,90.1,96.3,95.1,93.8
9,50,AVB,Avalonbay Communities Inc.,0.37,0.07,-0.05,33.6,N/A,86.4,66.0,35.8,62.7


In [131]:
# Script para eliminar filas axis=0 o columnas axis=1
#final_dataframe.drop(['12'], axis=1)


In [133]:
# Ordenamos el dataframe segun el campo Media
final_dataframe.sort_values('Media', ascending = False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,index,Ticker,Descripción,Rentab_1año,Rentab_6m,Rentab_1m,PER,n_acciones_a_comprar,Percentil_1año,Percentil_6meses,Percentil_1mes,Media
4,54,AXP,American Express Co.,0.50,0.23,0.21,9.7,N/A,94.4,91.4,100.0,95.3
8,12,ADM,Archer Daniels Midland Co.,0.41,0.31,0.09,8.0,N/A,90.1,96.3,95.1,93.8
7,5,ABBV,Abbvie Inc,0.45,0.26,0.07,34.1,N/A,91.4,93.8,92.0,92.4
5,19,AIG,American International Group Inc,0.50,0.19,0.05,2.9,N/A,94.4,87.7,89.5,90.5
21,67,BKR,Baker Hughes Co,0.29,0.47,0.12,-70.2,N/A,73.5,98.8,98.8,90.4
0,43,APA,APA Corporation,0.68,1.04,-0.00,19.8,N/A,100.0,100.0,69.1,89.7
1,38,ANET,Arista Networks Inc,0.63,0.38,0.00,23.4,N/A,98.8,97.5,69.1,88.5
14,6,ABC,Amerisource Bergen Corp.,0.36,0.20,0.07,18.9,N/A,82.1,89.5,92.0,87.9
3,40,ANTM,Anthem Inc,0.55,0.20,-0.00,9.1,N/A,96.3,89.5,69.1,85.0
2,55,AZO,Autozone Inc.,0.60,0.18,-0.01,17.7,N/A,97.5,86.4,59.9,81.3


In [134]:
# Find el proyecto, basta ahora exportarlo a un Excel
final_dataframe.to_excel('portafolio_equitativo.xlsx')
print('Fichero excel exportado con éxito. Mira en la carpeta "sample_data"')

Fichero excel exportado con éxito. Mira en la carpeta "sample_data"
